In [1]:
import pandas as pd
import numpy as np
import tkinter as tk
from tkinter import filedialog
from os import listdir
from os.path import isfile, join
from pprint import pprint
from pandas import ExcelWriter
'''

# Create Tk root
tk_root = tk.Tk()
# Hide the main window
tk_root.withdraw()
tk_root.call('wm', 'attributes', '.', '-topmost', True)
file_dir = filedialog.askdirectory(initialdir='./')
file_list =  [f for f in listdir(file_dir) if isfile(join(file_dir, f)) and f.endswith('.xlsx')and f.__contains__('8')]

i = 0
single = pd.DataFrame()
for f in file_list:    
    data = pd.read_excel(file_dir + '/' + f, header=0)
    if i == 0:
        single = pd.DataFrame(data= data['Cumulative Frame Time / s'])
        double = pd.DataFrame(data= data['Cumulative Frame Time / s'])
        column_name = ['Cumulative Frame Time / s']
    single_rename = pd.DataFrame.rename(data['Single Normalization'], columns=[f])
    single = pd.concat([single,single_rename], axis=1)
    double = pd.concat([single,pd.DataFrame(data= data['Double Normalization'])], axis=1)

    break

excel_path = file_dir + '/' + 'concatenatedResults.xlsx'


with ExcelWriter(excel_path) as writer:        
    single.to_excel(writer,sheet_name='Single Normalized')  
    double.to_excel(writer,sheet_name='Double Normalized')   


print('wrote results to ' + excel_path)
'''
#pprint(single_dict)

"\n\n# Create Tk root\ntk_root = tk.Tk()\n# Hide the main window\ntk_root.withdraw()\ntk_root.call('wm', 'attributes', '.', '-topmost', True)\nfile_dir = filedialog.askdirectory(initialdir='./')\nfile_list =  [f for f in listdir(file_dir) if isfile(join(file_dir, f)) and f.endswith('.xlsx')and f.__contains__('8')]\n\ni = 0\nsingle = pd.DataFrame()\nfor f in file_list:    \n    data = pd.read_excel(file_dir + '/' + f, header=0)\n    if i == 0:\n        single = pd.DataFrame(data= data['Cumulative Frame Time / s'])\n        double = pd.DataFrame(data= data['Cumulative Frame Time / s'])\n        column_name = ['Cumulative Frame Time / s']\n    single_rename = pd.DataFrame.rename(data['Single Normalization'], columns=[f])\n    single = pd.concat([single,single_rename], axis=1)\n    double = pd.concat([single,pd.DataFrame(data= data['Double Normalization'])], axis=1)\n\n    break\n\nexcel_path = file_dir + '/' + 'concatenatedResults.xlsx'\n\n\nwith ExcelWriter(excel_path) as writer:        

In [2]:
#!python 3.9.7

from skimage import io
import tkinter as tk
from tkinter import filedialog, messagebox
import matplotlib.pyplot as plt
from os import listdir, _exit
from os.path import isfile, join
import logging, sys, time
logging.basicConfig(stream=sys.stderr, level=logging.INFO)


from threading import Thread
from pprint import pprint

# def checkFileListOverwrite(file_dir):
#     file_list = [f for f in listdir(file_dir) if isfile(join(file_dir, f)) and f.endswith('.tif')]
#     xlsx_files = [f.removesuffix('.xlsx') for f in listdir(file_dir) if isfile(join(file_dir, f)) and f.endswith('.xlsx')]
#     result = messagebox.askyesnocancel('Old processing files found!','Would you like to overwrite them?')
#     if result:  
#         return file_list
#     elif result == False:
#         return [t for t in file_list if t.removesuffix('.tif') not in xlsx_files]
#     else:
#         return None

# # Create Tk root
# tk_root = tk.Tk()
# # Hide the main window
# tk_root.withdraw()
# tk_root.call('wm', 'attributes', '.', '-topmost', True)
# file_dir = filedialog.askdirectory(initialdir='./')
# file_list = checkFileListOverwrite(file_dir)
# pprint(file_list)

In [3]:
# -*- coding: utf-8 -*-
"""
Created on Tue Nov  1 16:09:06 2022

@author: damlatetiker
"""
import numpy as np
from skimage import filters, morphology, measure
import copy
import matplotlib.pyplot as plt
import pandas as pd
from PIL import Image
from PIL.TiffTags import TAGS

def detect_frap(curr_img):
    # this function determines the last frame before and first frame after bleaching by:
    # defining a rough estimate for the bleached condenste via thresholding (li's method is used) the first frame
    # meanming the intensity over all pixels in this ROI over the whole time series
    # determining the sudden drop in the intensity
    # curr_img should be an array of M x N x K with M indexing the time
    
    # Get the first frame from the image stack and determine the ideal threshold with li's method
    first_frame = curr_img[0].astype(float)
    thresh_li = filters.threshold_li(first_frame)
    
    # Create a mask where foreground is True and background is False by thresholding
    mask = first_frame > thresh_li
    # Remove objects smaller than a given size (for now 20 pixels) to get rid of other structures than the desired condenste
    mask = morphology.remove_small_objects(mask, 20)
    
    # initialize a list that would hold the mean of pixel intensities within the ROI over the time series
    int_seq = []
    for i in range(len(curr_img)):
        int_seq.append(np.mean(np.where(mask==True, curr_img[i], 0)))
    # Change the datatype of the intensity sequence to float, otherwise there are problems
    int_seq = np.array(int_seq).astype(float)
    # Take the 'derivative' of the intensity sequence, it should be 0 eg. constant function except for one very small number, the drop of intensity
    der = np.diff(int_seq)
    # Find where the derivative has a minimum, this is the index of the frame before bleaching
    ind_before = np.argmin(der)
    ind_after = ind_before+1
    
    return ind_before, ind_after

def refine_ROIs(curr_img, ind_before, ind_after):
    # Refine the ROI by taking the average of all frames up to bleaching and thresholding this average image
    # one needs to take ind_before+1 since indexing starts at 0
    frames_before = curr_img[:ind_before]
    img_avg = np.mean(frames_before, axis=0)
    
    # Determine the ideal threhold for the average image with li's method and apply it to get a mask
    thresh_li1 = filters.threshold_li(img_avg)
    mask_roi1 = img_avg>thresh_li1
    mask_roi1 = morphology.remove_small_objects(mask_roi1, 30)
    
    # get the first frame after bleaching
    first_after = curr_img[ind_after].astype(float)
    
    # Determine the ideal threhold for the first frame after bleaching with li's method and apply it to get a mask
    thresh_li2 = filters.threshold_li(first_after)
    mask_int = first_after>thresh_li2
    mask_int = morphology.remove_small_objects(mask_int, 30)
    
    # The second ROI (bleaching site) are the pixels that are true in the first mask and false in the second one
    mask_roi2 = copy.deepcopy(mask_roi1)
    mask_roi2[mask_int==True] = False
    
    # Make sure you have only one region, as the process above may introduce some problems
    label_image = measure.label(mask_roi2)
    if np.amax(label_image) != 1:
        logging.info('More than one region found')
        pass
        ## TODO try using the biggest ROI in set


        #mask_roi1 = label_image[label_image == 1]
        #mask_roi2 = label_image[label_image == 2]
        #raise NotImplementedError('The mask for the bleached area should have 1 label but instead has {}'.format(np.amax(label_image)))
        # It is not yet implemented how to deal with such an incorrect segmentation
    return mask_roi1, mask_roi2

def refine_ROIs2(curr_img, ind_before, ind_after):
    print('The second method for ROI calculation is not implemented yet')
    return 0, 0

def get_background_(mic_img, area_cond):
    avg_img = np.mean(mic_img, axis=0)
    thresh_mean = filters.threshold_mean(avg_img)
    backg_mask = avg_img<thresh_mean
    backg_size = int(np.sqrt(3*area_cond))
    
    return
def parse_tif_metadata(file_path):
    with Image.open(file_path) as img:
        meta_dict = {TAGS[key] : img.tag[key] for key in img.tag.keys()}    
    desc = meta_dict['ImageDescription'][0]
    desc = desc.split()
    keys = [d.split('=')[0] for d in desc]
    vals = [d.split('=')[1] for d in desc]
    for i in [1,2,4,6,7]:
        vals[i] = float(vals[i])
    vals[5] = bool(vals[5])
    return {keys[i]: vals[i] for i in range(len(keys))}

def export_intensity(curr_img, mask_roi1, mask_roi2, file_path):
    # Total area
    img_area_1 = copy.deepcopy(curr_img)
    img_area_1[:, mask_roi1==False] = 0
    area_total_ = np.mean(mask_roi1)
    area_total = np.ones(len(curr_img))*area_total_
    mean_intensity_whole_area = np.sum(np.sum(img_area_1, axis=1), axis=1)/area_total_
    
    # Bleached area
    img_area_2 = copy.deepcopy(curr_img)
    img_area_2[:, mask_roi2==False] = 0
    area_bleached_ = np.sum(mask_roi2)
    area_bleached = np.ones(len(curr_img))*area_bleached_
    mean_intensity_bleached_area = np.sum(np.sum(img_area_2, axis=1), axis=1)/area_bleached_
    
    # Unbleached area
    mask_roi3 = copy.deepcopy(mask_roi1)
    mask_roi3[mask_roi2==True] = False
    img_area_3 = copy.deepcopy(curr_img)
    img_area_3[:, mask_roi3==False] = 0
    area_unbleached_ = np.sum(mask_roi3)
    area_unbleached = np.ones(len(curr_img))*area_unbleached_
    mean_intensity_unbleached_area = np.sum(np.sum(img_area_3, axis=1), axis=1)/area_unbleached_
    
    ## TODO: Parse frame times and write cumulative frame time
    metadata = parse_tif_metadata(file_path)
    frametimes = [metadata['finterval']*i for i,t in enumerate(area_bleached)]

    ## TODO: Implement prebleach series length
 
    # doubleNoormlization = [(TotalROI_1-20 - Bg_1-20) * (TotalROI_slice - Bg_slice)] / [(Bleached_1-20 - Bg_1-20) * (Bleached_slice - Bg_slice)] 
    # singleNormalization = [Bleached_slice - Bg_slice] / [Bleach_1-20 - Bg_1-20]
    

    last_pre_frame = detect_frap(curr_img)[0]

    mean_intensity_total_ROI= mean_intensity_bleached_area + mean_intensity_unbleached_area
    mean_intensity_background = mean_intensity_whole_area - mean_intensity_total_ROI


    doubleNorm = np.array([
        ((np.mean(mean_intensity_total_ROI[:last_pre_frame]) - np.mean(mean_intensity_background[:last_pre_frame]))* 
        (mean_intensity_total_ROI[_] - mean_intensity_background[_])) / 
        ((np.mean(mean_intensity_bleached_area[:last_pre_frame]) - np.mean(mean_intensity_background[:last_pre_frame])) * 
        (mean_intensity_bleached_area[_] - mean_intensity_background[_])) for _,e in enumerate(mean_intensity_whole_area)])

    singleNorm = [
        (mean_intensity_bleached_area[_]- mean_intensity_background[_]) / 
        (np.mean(mean_intensity_bleached_area[:last_pre_frame]) - np.mean(mean_intensity_background[:last_pre_frame]))
         for _,e in enumerate(mean_intensity_whole_area)]
    
    #TODO New script: concatenate double and single norm from whole FRAP experiment, first colum frame time
    
    results = {'Area(all)':area_total,
               'Mean_intensity(all)': mean_intensity_whole_area,
               'Area(bleached)': area_bleached,
               'Mean_intensity(bleached)': mean_intensity_bleached_area,
               'Area(unbleached)': area_unbleached,
               'Mean_intensity(unbleached)': mean_intensity_unbleached_area,
               'Cumulative Frame Time / s': frametimes,
               'Double Normalization': doubleNorm,
               'Single Normalization': singleNorm}
    
    results = pd.DataFrame(data=results)
    excel_path = file_path[:-4]+'.xlsx'
    logging.info('wrote results to' + excel_path)
    results.to_excel(excel_path)   
    return
    
# -*- coding: utf-8 -*-
"""
Created on Tue Nov  1 11:10:18 2022

@author: damlatetiker
"""


import numpy as np
import matplotlib.pyplot as plt
from skimage import io, measure, filters
from matplotlib.widgets import Slider, RectangleSelector, Button
import tkinter as tk
from tkinter import filedialog, messagebox
from os import listdir, _exit
from os.path import isfile, join
import logging, sys, time
import threading
logging.basicConfig(stream=sys.stderr, level=logging.INFO)
%matplotlib qt


class ImageVisualizer:
    def __init__(self): 
        self.display_directory_dialog()
        self.file_path = self.file_dir + '/' + self.file_list.pop(0)                        
        self.createFigure(self.file_path)
        self.update()
        self.connect()
        return


    # def __init__(self, path, filelist= []):
    #     if filelist.len() == 0 and path.endswith('.tif'):
    #         self.file_path = path
    #     elif filelist.len() > 0:
    #         self.file_dir = path
    #         self.file_list = filelist
    #         self.file_path = self.file_dir + '/' + self.file_list.pop(0)
    #     else:
    #         logging.debug(path)
    #         raise Exception('Unknown file path')

    #     self.createFigure()
    #     self.update()
    #     self.connect()
    #     return

    def set_path(self, path):        
        self.file_path = path  
        
    def set_dir(self, dir):        
        self.file_dir = dir  

 
    def checkFileListOverwrite(self,file_dir):
        file_list = [f for f in listdir(file_dir) if isfile(join(file_dir, f)) and f.endswith('.tif')]
        xlsx_files = [f.removesuffix('.xlsx') for f in listdir(file_dir) if isfile(join(file_dir, f)) and f.endswith('.xlsx')]
        result = messagebox.askyesnocancel('Old processing files found!','Would you like to overwrite them?')
        if result:  
            return file_list
        elif result == False:
            return [t for t in file_list if t.removesuffix('.tif') not in xlsx_files]
        else:
            return None

    ## TODO Dialog full/part
    ## TODO Set slider slice to last value before dark --> based on user input OR remember from last slice
    def display_directory_dialog(self):
        # Create Tk root
        tk_root = tk.Tk()
        # Hide the main window
        tk_root.withdraw()
        tk_root.call('wm', 'attributes', '.', '-topmost', True)
        file_dir = filedialog.askdirectory(initialdir='./')
        self.file_list = self.checkFileListOverwrite(file_dir)
        self.set_dir(file_dir)
        return

    def createFigure(self, path):
        self.fig, self.ax = plt.subplots(figsize=(9,6))  
        self.mic_img = io.imread(path)
        axslid = self.fig.add_axes([0.3, 0.03, 0.4, 0.03])
        self.slider = Slider(ax=axslid, label='Slice', valmin=0, valmax=self.mic_img.shape[0]-1, valinit=0, valstep=1, initcolor='none', color='lightgrey')
        

        self.ax.set_aspect('equal')
        self.ax.set_title('use scroll wheel to navigate images')
        self.selector = RectangleSelector(self.ax,self.select_roi)
        
        self.curr_img = self.mic_img
        self.slices, rows, cols = self.mic_img.shape
        self.ind = 0

        self.im = self.ax.imshow(self.mic_img[self.ind, :, :], cmap='gray')#, vmax=2000)
        self.curr_img_bbox = [-0.5, 255.5, 255.5, -0.5]
        self.update()
        return
        
    def load_next_stack(self, event):    
        logging.debug(self.file_list)       
        if self.file_list.__len__() > 0:  
            self.export(None)            
            self.file_path = self.file_dir + '/' + self.file_list.pop(0)  
            plt.close('all')
            self.createFigure(self.file_path) 
            self.update()
            self.connect()           
            return
        else:
            messagebox.showinfo('Finished!')
            plt.close()
            _exit() 
    
    def connect(self):
        self.fig.canvas.mpl_connect('scroll_event', self.on_scroll)
        self.fig.canvas.mpl_connect('key_press_event', self.on_press)
        self.slider.on_changed(self.update_from_slider)
        return
    
    def connect2(self):
        self.but1.on_clicked(self.ROI_method1)
        self.but2.on_clicked(self.ROI_method2)
        self.but3.on_clicked(self.export)
        self.but4.on_clicked(self.load_next_stack)
        return
    
    def on_scroll(self, event):
        #print("%s %s" % (event.button, event.step))
        if event.button == 'up':
            self.ind = (self.ind + 1) % self.slices
        else:
            self.ind = (self.ind - 1) % self.slices
        self.slider.set_val(self.ind)
        self.update()
        return
        
    def on_press(self, event):
        if event.key == 'up' or event.key == 'right':
            self.ind = (self.ind + 1) % self.slices
        elif event.key == 'down'or event.key == 'left':
            self.ind = (self.ind - 1) % self.slices
        elif event.key == 'shift+alt':
            self.curr_img = self.mic_img
            self.curr_img_bbox = [-0.5, 255.5, 255.5, -0.5]
            self.ax.set_title('use scroll wheel to navigate images')
        elif event.key == 'enter':
            self.ind_before, self.ind_after = detect_frap(self.curr_img)
            self.ax.set_title('Now showing the ROIs calculated with method 1.\n'+
                    'To swith press button ROI method 2.\nTo see the background ROI press shift alt (not implemented yet)\n'+
                    'To export press button Export\nTo export and load the next stack, press Next Stack\n')
            ax_met1 = self.fig.add_axes([0.8, 0.8, 0.1, 0.06])
            ax_met2 = self.fig.add_axes([0.8, 0.7, 0.1, 0.06])
            ax_met3 = self.fig.add_axes([0.8, 0.6, 0.1, 0.06])            
            ax_met4 = self.fig.add_axes([0.8, 0.5, 0.1, 0.06])
            self.but1 = Button(ax_met1, 'ROI method 1')
            self.but2 = Button(ax_met2, 'ROI method 2 \n not implem. ')
            self.but3 = Button(ax_met3, 'Export')
            self.but4 = Button(ax_met4, 'Next Stack')
            # mask_roi1 is the total area, mask_roi2 is the bleached area
            # To get the unbleached area (necessary for exporting, see the function export_intensity) we subtract the second mask from the first one
            self.mask_roi1, self.mask_roi2 = refine_ROIs(self.curr_img, self.ind_before, self.ind_after)
            self.area_cond = np.sum(self.mask_roi1)
            self.get_background()
            contour1 = measure.find_contours(self.mask_roi1 == 1)[0]
            contour2 = measure.find_contours(self.mask_roi2 == 1)[0]
            bbox = self.curr_img_bbox
            self.ax.plot(contour1[:,1]+bbox[0]+0.5, contour1[:,0]+bbox[3]+0.5, c='r')
            self.ax.plot(contour2[:,1]+bbox[0]+0.5, contour2[:,0]+bbox[3]+0.5, c='b')
            self.connect2()
        self.slider.set_val(self.ind)
        self.update()
        return
  
    
    def ROI_method1(self, event):
        self.ax.set_title('Now showing the ROIs calculated with method 1.\nTo swith press button ROI method 2.\nTo see the background ROI press shift alt (not implemented yet)\nTo export press button Export')
        self.mask_roi1, self.mask_roi2 = refine_ROIs(self.curr_img, self.ind_before, self.ind_after)
        self.draw_ROI(self.mask_roi1, self.mask_roi2)
        return
    
    def ROI_method2(self, event):
        self.ax.set_title('Now showing the ROIs calculated with method 2.\nTo swith press button ROI method 1.\nTo see the background ROI press shift alt (not implemented yet)\nTo export press button Export')
        self.mask_roi1, self.mask_roi2 = refine_ROIs2(self.curr_img, self.ind_before, self.ind_after)
        self.draw_ROI(self.mask_roi1, self.mask_roi2)
        return
    
    def get_background(self):
        get_background_(self.mic_img, self.area_cond)
        return
        
    def export(self, event):
        export_intensity(self.curr_img, self.mask_roi1, self.mask_roi2, self.file_path)
        return
    
    def draw_ROI(self, mask1, mask2):
        self.ax.lines=[]
        contour1 = measure.find_contours(mask1 == 1)[0]
        contour2 = measure.find_contours(mask2 == 1)[0]
        bbox = self.curr_img_bbox
        ### TODO: Create polygons instead of plot
        self.ax.plot(contour1[:,1]+bbox[0]+0.5, contour1[:,0]+bbox[3]+0.5, c='r')
        self.ax.plot(contour2[:,1]+bbox[0]+0.5, contour2[:,0]+bbox[3]+0.5, c='b')
        self.im.set_extent(bbox)
        return  
            
    def update(self):
        self.im.set_extent(self.curr_img_bbox)
        self.im.set_data(self.curr_img[self.ind, :, :])
        self.ax.set_aspect('equal')
        self.ax.set_ylabel('slice %s' % self.ind)
        self.im.axes.figure.canvas.draw()
        self.fig.canvas.draw_idle()
        return

    def update_from_slider(self, val):
        self.ind = val
        self.update()
        return
    
    def select_roi(self, eclick, erelease):
        self.curr_img = self.mic_img[:, int(eclick.ydata):int(erelease.ydata), int(eclick.xdata):int(erelease.xdata)]
        self.curr_img_bbox = [int(eclick.xdata)-0.5, int(erelease.xdata)-0.5, int(erelease.ydata)-0.5, int(eclick.ydata)-0.5]
        self.ax.set_title('Are you happy with the current ROI?\nTo reset press shift and alt\nTo continue with automated selection press enter')
        self.update()
        return

vis = ImageVisualizer()
